In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk import Address, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
esdt_contract = Address.new_from_bech32("erd1qqqqqqqqqqqqqqqpqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqzllls8a5w6u")
router_contract = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgqq66xk9gfr4esuhem3jru86wg5hvp33a62jps2fy57p")
simple_lock_contract = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgq6nu2t8lzakmcfmu4pu5trjdarca587hn2jpsyjapr5")
egld_simple_lock_contract = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgq6nu2t8lzakmcfmu4pu5trjdarca587hn2jpsyjapr5")

In [ ]:
pair_contract: Address
paired_token = "USDC-c76f1f"
wegld_token = "WEGLD-bd4d79"
locked_token = "LKESDT-fc19ba"
egld_locked_token = "LKESDT-fc19ba"
issued_token: str
pair_token: str
locked_lp_token: str

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
owner = env.OWNER

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
owner_signer = UserSigner.from_pem_file(Path(env.OWNER_PEM))

configure router

usdc pairs

In [ ]:
from multiversx_sdk.converters.transactions_converter import TransactionsConverter

builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="configEnableByUserParameters",
    caller=owner,
    call_arguments=[paired_token, locked_token, 495000000, 30],
    gas_limit=30000000
)

tx = builder.build()

converter = TransactionsConverter()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
from multiversx_sdk import TransactionComputer

user_nonce_holder = AccountNonceHolder(proxy.get_account(owner).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = owner_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = proxy.send_transaction(tx)
print(hash)

egld pairs

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="addCommonTokensForUserPairs",
    caller=owner,
    call_arguments=[wegld_token],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(owner).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = owner_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="configEnableByUserParameters",
    caller=owner,
    call_arguments=[wegld_token, egld_locked_token, 1000000000000000000, 30],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(owner).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx.signature = owner_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

issue token

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=esdt_contract,
    function_name="issue",
    caller=user,
    call_arguments=["TOKEN3", "TOK3", 1000000000000000000000000000, 18, "canMint", "true", "canBurn", "true", "canPause", "true", "canFreeze", "true", "canWipe", "true", "canChangeOwner", "true", "canUpgrade", "true", "canAddSpecialRoles", "true"],
    gas_limit=100000000,
    value=TokenPayment.egld_from_integer(50000000000000000)
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
issue_event = tx_data.logs.find_first_or_none_event("issue")
issued_token = str(issue_event.topics[0])
print(issue_event.topics[0].raw)
print("Issued token:", issued_token)

create pair USDC

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="createPair",
    caller=user,
    call_arguments=[issued_token, paired_token, user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("SCDeploy")
topic = event.topics[0]
pair_contract = Address.new_from_hex(topic.hex(), "erd")
print("Deployed pair:", pair_contract.to_bech32())

issue lp token

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="issueLpToken",
    caller=user,
    call_arguments=[pair_contract, "TKNUSDCLP", "TKNUSDC"],
    gas_limit=100000000,
    value=TokenPayment.egld_from_integer(50000000000000000)
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
# event = tx_data.logs.find_first_or_none_event("issue")
# pair_token = str(event.topics[0])
# print("LP token:", pair_token)

set local role

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="setLocalRoles",
    caller=user,
    call_arguments=[pair_contract],
    gas_limit=100000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

add initial liquidity

In [ ]:
transfers = [
    TokenPayment.fungible_from_amount(issued_token, 1050, 18),
    TokenPayment.fungible_from_amount(paired_token, 1050, 6)
]

builder = ContractCallBuilder(
    config,
    contract=pair_contract,
    function_name="addInitialLiquidity",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("ESDTTransfer")
pair_token = str(event.topics[0])
pair_token_amount = int(event.topics[2].hex(), 16)
print(f"LP token: {pair_token} amount {pair_token_amount}")

lock tokens

In [ ]:
transfers = [
    TokenPayment.fungible_from_integer(pair_token, pair_token_amount, 18)
]

current_epoch = proxy.get_network_status().current_epoch

builder = ContractCallBuilder(
    config,
    contract=simple_lock_contract,
    function_name="lockTokens",
    caller=user,
    call_arguments=[current_epoch+30],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("ESDTNFTTransfer")
locked_lp_token = str(event.topics[0])
locked_lp_token_nonce = int(event.topics[1].hex(), 16)
locked_lp_token_amount = int(event.topics[2].hex(), 16)
print(f"Locked LP token: {locked_lp_token} nonce {locked_lp_token_nonce} amount {pair_token_amount}")

swap enable by user

In [ ]:
transfers = [
    TokenPayment.meta_esdt_from_integer(locked_lp_token, locked_lp_token_nonce, 495000000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="setSwapEnabledByUser",
    caller=user,
    call_arguments=[pair_contract],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
event = tx_data.logs.find_first_or_none_event("signalError")
completed = tx_data.logs.find_first_or_none_event("completedTxEvent")
if event:
    print("Transaction failed")
elif completed:
    print("all good")
else:
    print("Probably not finished yet...")

Create pairs EGLD

In [ ]:
wegld_token = "TOK3-39c76b"

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="createPair",
    caller=user,
    call_arguments=[issued_token, wegld_token, user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("SCDeploy")
topic = event.topics[0]
pair_contract = Address.from_hex(topic.hex(), "erd")
print("Deployed pair:", pair_contract.bech32())

issue lp token

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="issueLpToken",
    caller=user,
    call_arguments=[pair_contract, "TKNEGLDLP", "TKNEGLD"],
    gas_limit=100000000,
    value=TokenPayment.egld_from_integer(50000000000000000)
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)

print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
# event = tx_data.logs.find_first_or_none_event("issue")
# pair_token = str(event.topics[0])
# print("LP token:", pair_token)

set local role

In [ ]:
builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="setLocalRoles",
    caller=user,
    call_arguments=[pair_contract],
    gas_limit=100000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

add initial liquidity

In [ ]:
transfers = [
    TokenPayment.fungible_from_amount(issued_token, 1.1, 18),
    TokenPayment.fungible_from_amount(wegld_token, 1.1, 18)
]

builder = ContractCallBuilder(
    config,
    contract=pair_contract,
    function_name="addInitialLiquidity",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("ESDTTransfer")
pair_token = str(event.topics[0])
pair_token_amount = int(event.topics[2].hex(), 16)
print(f"LP token: {pair_token} amount {pair_token_amount}")

lock tokens

In [ ]:
egld_simple_lock_contract = Address.from_bech32("erd1qqqqqqqqqqqqqpgq3gpzrd6854znu0xecd7k2tqxnsc3u5pe2jpsqnq5du")

In [ ]:
transfers = [
    TokenPayment.fungible_from_integer(pair_token, pair_token_amount, 18)
]

current_epoch = proxy.get_network_status().current_epoch

builder = ContractCallBuilder(
    config,
    contract=egld_simple_lock_contract,
    function_name="lockTokens",
    caller=user,
    call_arguments=[current_epoch+30],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("ESDTNFTTransfer")
locked_lp_token = str(event.topics[0])
locked_lp_token_nonce = int(event.topics[1].hex(), 16)
locked_lp_token_amount = int(event.topics[2].hex(), 16)
print(f"Locked LP token: {locked_lp_token} nonce {locked_lp_token_nonce} amount {pair_token_amount}")

swap enable by user

In [ ]:
transfers = [
    TokenPayment.meta_esdt_from_amount(locked_lp_token, locked_lp_token_nonce, 1, 18)
]

builder = ContractCallBuilder(
    config,
    contract=router_contract,
    function_name="setSwapEnabledByUser",
    caller=user,
    call_arguments=[pair_contract],
    gas_limit=50000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
hash = proxy.send_transaction(tx)
print(hash)

In [ ]:
tx_data = proxy.get_transaction(hash)
event = tx_data.logs.find_first_or_none_event("signalError")
completed = tx_data.logs.find_first_or_none_event("completedTxEvent")
if event:
    print("Transaction failed")
elif completed:
    print("all good")
else:
    print("Probably not finished yet...")